In [1]:
from algorithms.statics import *
from algorithms.pkgs import MongoClient, csv
seen_id_pairs = set()
client = MongoClient(DB_CONNECTION)

In [3]:
def ingest(src_file):
    client.course_evals.entries.drop()
    with open(src_file,encoding='latin-1') as csvfile:
        reader = csv.reader(csvfile,delimiter=',')
        row_num = 0
        for r in reader:
            row_num += 1
            if row_num == 1: continue
            entry = {key : r[index] for index,key in enumerate(HEADER_ROW) if key in RELEVANT_ROWS}
            tokens = entry["RESPONSE_TEXT"].split()
            id_pair = tuple((entry['RESPONSE_UNIQUE_ID'],entry['QUESTION_ID'])) # de-duplication
            if len(tokens) > 1 and id_pair not in seen_id_pairs:
                print(row_num)
                seen_id_pairs.add(id_pair)
                client.course_evals.entries.insert_one(entry)

In [ ]:
ingest(src_file='../res/course-evals.csv')